## Final Project Submission

Please fill out:
* Student name: Eduardo Calzadilla
* Student pace: self paced 
* Scheduled project review date/time: 
* Instructor name: Jeff Herman
* Blog post URL:


# Movie genre and title recommendations

#### Datasets: 
    1. Box Office Mojo
    2. IMBD
    3. Rotten Tomatoes
    4. TheMovieDB.org



## Contents

**A. [Exploratory Data Analysis](#EDA)** <br>
  1. [Initial exploration and cleanup](#Iec)<br>
  2. [Exploratory visualizations](#Ev)<br>
  3. [Transformations and joins](#Tj)<br>
  
**B.**
 

## Introduction & Summary

Throughout this notebook we will be exploring three datasets as highlighted above, in order to spot trend that could be indicative of higher succes capacity and return on investment (ROI). 

We will be focusing on the following four null hypothesese:
  1. Movies with higher ratings do not perform significantly better in terms gross income and ROI.
  2. All genres perform equally in terms of ratings, gross income and ROI.
  3. There are no words that are indicative of higher performance in terms of ratings and ROI.
  4. There has been no change in terms of competition (number of movies coming out) 4 year prior 2015 and 4 year after 2015. 

We have found that there are significant similarities between median ratings and median performance. etc...

In [ ]:
# Importing necessay packages
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline




In [ ]:
# Your code here - remember to use markdown cells for comments as well!